In [130]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

from settings.key import GPT_API_KEY
openai.api_key = GPT_API_KEY
from LLMSearch.AutoDataset.GPTQuestions import gpt_function_call

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [131]:

from LLMSearch.AnswerBot import AnswerBot,parse_answer
from settings.key import GPT_API_KEY, DEEPL_API_KEY
from LLMSearch.GPTQuery import GPTQuery
from LLMSearch.BM25DB import BM25DB
from LLMSearch.SQLTextDB import SQLTextDB
from LLMSearch.Embedding.SBERTFineTuneEmbedding import SBERTFineTuneEmbedding
from LLMSearch.VectorSearcher import VectorSearcher

embedder=SBERTFineTuneEmbedding()
searcher=VectorSearcher(embedder,chunk_size_limit=500)


#bm=BM25DB(initiate=True)
#searcher=SQLTextDB()

# initiate bot module
bot1 = AnswerBot(query_module=GPTQuery(GPT_API_KEY),
                searcher=searcher,
                DEEPL_API_KEY=DEEPL_API_KEY,
                )

[nltk_data] Downloading package punkt to /home/kh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmplq527ht7
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmplq527ht7/_remote_module_non_scriptable.py
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [188]:
from LLMSearch.DocSplitter import split_text,remove_stopwords
path=bot1.searcher.path_list[-1]
path_list=list(set(bot1.searcher.path_list))
path_list=[path for path in path_list if path.find("koko")==-1]


In [191]:
path_list=path_list[:10]

In [171]:

import shutil
count=0
for path in path_list:
    with open(path, 'r') as f:
        text = f.read()
    chunk_list = split_text(text, bot1.searcher.chunk_size_limit)

    # remove stopwords
    chunk_list = [remove_stopwords(chunk) for chunk in chunk_list]


    for text in chunk_list:
        questions=gpt_function_call(text)

        out_path="UserData/meta/qa/qa.txt"
        for q in questions:
            with open(out_path, 'a') as f:
                f.write(path)
                f.write("|||||")
                f.write(questions[q])
                f.write("|||||")
                f.write(text)
                f.write("\n")
        print(questions)
    #コピーを作成してバックアップ
    shutil.copyfile(out_path, f"UserData/meta/qa/qa_backup{count}.txt")
    count+=1

{'q1': 'ポリマーの結晶性が減少するのは何が原因ですか？',
 'q2': '高いシンジオタクチック立体規則性を持つ主鎖を持つポリマーの合成には何を使用しましたか？',
 'q3': 'くし状ポリマーを合成するために使用した触媒は何ですか？',
 'q4': 'グラフト共重合体を合成するために使用した触媒は何ですか？',
 'q5': 'マクロモノマーとスチレンの共重合を行うために使用した触媒は何ですか？',
 'q6': '3元共重合体の合成にどのような手法を使用しましたか？',
 'q7': 'ABAブロック共重合体の合成にどのような手法を使用しましたか？',
 'q8': 'メタロセン触媒はどのような重合の可能性を与えますか？'}

'UserData/meta/qa/qa_backup.txt'